# Access Logs

MNs log object access and other object events. These logs are collected and aggregated by CNs. Depending on CN and MN policies, the logs can be retrieved from CNs and or MNs.

In general, logs are access controlled, and can only be retrieved by DataONE subjects that have elevated access to the related objects. In particular, this means that giving public read access to an object does not cause its log to also become publicly accessible.

Logs are retrieved from the CN with

[CNCore.getLogRecords(session[, fromDate][, toDate][, event][, idFilter][, start][, count]) → Log](https://releases.dataone.org/online/api-documentation-v2.0/apis/CN_APIs.html#CNCore.getLogRecords)

and from the MN with

[MNCore.getLogRecords(session[, fromDate][, toDate][, event][, idFilter][, start=0][, count=1000]) → Log](https://releases.dataone.org/online/api-documentation-v2.0/apis/MN_APIs.html#MNCore.getLogRecords)

Logs from the MN will contain only records captured on that MN, while logs retrieved from the CN will contain records potentially aggregated across multiple MNs that may hold replicas of the objects.

`getLogRecords()` returns a [Log](https://releases.dataone.org/online/api-documentation-v2.0/apis/Types.html#Types.Log) instance which contain a sequence of [LogEntry](https://releases.dataone.org/online/api-documentation-v2.0/apis/Types.html#Types.LogEntry) instances.

The types of events that are logged include `create`, `read`, `update`, `delete`, `replicate`, `synchronization_failed`, and `replication_failed`.

The client and user information that can potentially be logged for events includes `ipAddress`, `userAgent`, `subject`, `dateLogged` and `nodeIdentifier`.


## Python Client Library example

### Create a CN or MN client

In this example, we create a CN client. We authenticate by supplying a certificate. This makes logs available for objects for which the subject(s) in the the certificate have elevated access.

In [10]:
import d1_client.cnclient_2_0

cn_client = d1_client.cnclient_2_0.CoordinatingNodeClient_2_0(
  'https://cn-stage.test.dataone.org/cn',
  cert_pem_path='./urn_node_myMemberNode.pem',
  cert_key_path='./urn_node_myMemberNode.key',
)

### Call getLogRecords()

As there can potentially be a large number of log records available, the result is returned in batches, as designated by the `start` and `count` parameters. The total number of records is available in the `total` member of the returned `Log` instance.

We also provide a [log record iterator](logRecordIterator) which enables more convenient retrieval of the results from `getLogRecords()`.

In [11]:
log_pyxb = cn_client.getLogRecords(start=0, count=10)

### Iterate over the results

Since we specified a count of 10 in the `getLogRecords()` call, we will receive at most 10 results.

Now we iterate over the sequence of `LogEntry` instances in the returned `Log` and print some selected values.

In [13]:
for log_entry_pyxb in log_pyxb.logEntry:
  print u'{} - {} - {} - {}'.format(
    log_entry_pyxb.identifier.value(),
    log_entry_pyxb.ipAddress,
    log_entry_pyxb.subject.value(),
    log_entry_pyxb.event,
)

testCnGetLogRecords_Aggregating_urnnodemnTestUIC_obj2 - 64.106.40.19 - CN=urn:node:cnStageUNM1,DC=dataone,DC=org - create
testCnGetLogRecords_Access_public_urnnodemnTestUIC - 64.106.40.19 - CN=urn:node:cnStageUNM1,DC=dataone,DC=org - create
testUpdateSystemMetadata_MutableAccessPolicy_20173214724315 - 64.106.40.19 - CN=testRightsHolder,DC=dataone,DC=org - create
testUpdateSystemMetadata_MutableAccessPolicy_20173214724315 - 64.106.40.19 - CN=testRightsHolder,DC=dataone,DC=org - updateSystemMetadata
testUpdateSystemMetadata_MutableAccessPolicy_20173214724315 - 128.111.54.76 - CN=urn:node:cnStageUCSB1,DC=dataone,DC=org - read
testUpdateSystemMetadata_MutableAccessPolicy_20173214724315 - 128.111.54.76 - CN=urn:node:cnStageUCSB1,DC=dataone,DC=org - read
testCnGetLogRecords_Access_public_urnnodemnTestUIC - 128.111.54.76 - CN=urn:node:cnStageUCSB1,DC=dataone,DC=org - read
testCnGetLogRecords_Access_public_urnnodemnTestUIC - 128.111.54.76 - CN=urn:node:cnStageUCSB1,DC=dataone,DC=org - read
tes